In [1]:
from pyomo.environ import (
    ConcreteModel, Set, Param, Var, NonNegativeReals, Binary, UnitInterval,
    Constraint, Objective, minimize, SolverFactory, value, Suffix
)

In [2]:
# ------------------------------------------------------------
# 1) Build model (covers: multi-period inventory + capacity + unmet + setup + Big-M + min-run)
# ------------------------------------------------------------
def build_model(data, relax_binaries_for_duals=False):
    m = ConcreteModel(name="MultiPeriod_SOP_Template")

    # ----------------------------
    # SETS
    # ----------------------------
    m.T = Set(initialize=data["T"], ordered=True)

    # ----------------------------
    # PARAMETERS
    # ----------------------------
    m.demand = Param(m.T, initialize=data["demand"])             # demand[t]
    m.cap    = Param(m.T, initialize=data["cap"])                # capacity[t]

    m.c_prod    = Param(initialize=data["c_prod"])               # unit production cost
    m.c_hold    = Param(initialize=data["c_hold"])               # unit holding cost
    m.c_unmet   = Param(initialize=data["c_unmet"])              # penalty per unit unmet
    m.c_setup   = Param(initialize=data["c_setup"])              # fixed setup cost per start
    m.min_run   = Param(initialize=data["min_run"])              # minimum run if setup=1
    m.init_inv  = Param(initialize=data["init_inv"])             # starting inventory

    # Big-M should be a *tight upper bound* on prod[t].
    # A safe simple bound: capacity[t]. (Often best!)
    # If you want even tighter: min(cap[t], sum future demand) etc.
    def M_init(m, t):
        return float(value(m.cap[t]))
    m.M = Param(m.T, initialize=M_init)

    # ----------------------------
    # DECISION VARIABLES
    # ----------------------------
    m.prod  = Var(m.T, domain=NonNegativeReals)                  # production
    m.inv   = Var(m.T, domain=NonNegativeReals)                  # end inventory
    m.unmet = Var(m.T, domain=NonNegativeReals)                  # unmet demand (lost sales)

    # setup: binary (or relaxed to [0,1] if we want duals)
    if relax_binaries_for_duals:
        m.setup = Var(m.T, domain=UnitInterval)                  # 0..1 relaxation
    else:
        m.setup = Var(m.T, domain=Binary)

    # ----------------------------
    # CONSTRAINTS
    # ----------------------------

    # 1) Capacity limit
    def cap_rule(m, t):
        return m.prod[t] <= m.cap[t]
    m.capacity_con = Constraint(m.T, rule=cap_rule)

    # 2) Inventory balance:
    # inv[t] = inv[t-1] + prod[t] - served[t]
    # served[t] = demand[t] - unmet[t]
    # => inv[t] = inv[t-1] + prod[t] - (demand[t] - unmet[t])
    # => inv[t] = inv[t-1] + prod[t] - demand[t] + unmet[t]
    T_list = list(data["T"])

    def inv_balance_rule(m, t):
        idx = T_list.index(t)
        inv_prev = m.init_inv if idx == 0 else m.inv[T_list[idx - 1]]
        return m.inv[t] == inv_prev + m.prod[t] - m.demand[t] + m.unmet[t]
    m.inv_balance = Constraint(m.T, rule=inv_balance_rule)

    # 3) Big-M linking: if setup[t] = 0 -> prod[t] must be 0
    # prod[t] <= M[t] * setup[t]
    def bigm_rule(m, t):
        return m.prod[t] <= m.M[t] * m.setup[t]
    m.link_setup = Constraint(m.T, rule=bigm_rule)

    # 4) Minimum run: if setup[t] = 1 -> prod[t] >= min_run
    # prod[t] >= min_run * setup[t]
    def minrun_rule(m, t):
        return m.prod[t] >= m.min_run * m.setup[t]
    m.min_run_con = Constraint(m.T, rule=minrun_rule)

    # ----------------------------
    # OBJECTIVE (Total cost)
    # ----------------------------
    def obj_rule(m):
        return sum(
            m.c_prod  * m.prod[t] +
            m.c_hold  * m.inv[t] +
            m.c_unmet * m.unmet[t] +
            m.c_setup * m.setup[t]
            for t in m.T
        )
    m.obj = Objective(rule=obj_rule, sense=minimize)

    # ----------------------------
    # DUALS (only meaningful when model is LP / continuous relaxation)
    # ----------------------------
    if relax_binaries_for_duals:
        m.dual = Suffix(direction=Suffix.IMPORT)

    return m


In [3]:
# ------------------------------------------------------------
# 2) Solve one scenario
# ------------------------------------------------------------
def solve_scenario(data, relax_for_duals=False, solver_name="glpk"):
    m = build_model(data, relax_binaries_for_duals=relax_for_duals)

    solver = SolverFactory(solver_name)
    results = solver.solve(m, tee=False)

    # Collect results
    out = {
        "total_cost": value(m.obj),
        "prod":  {t: value(m.prod[t])  for t in m.T},
        "inv":   {t: value(m.inv[t])   for t in m.T},
        "unmet": {t: value(m.unmet[t]) for t in m.T},
        "setup": {t: value(m.setup[t]) for t in m.T},
        "duals_capacity": None
    }

    # Shadow prices for capacity (dual of capacity constraint), only in relaxed model
    if relax_for_duals:
        out["duals_capacity"] = {t: m.dual.get(m.capacity_con[t], None) for t in m.T}

    return out


In [4]:
# ------------------------------------------------------------
# 3) Scenario loop (What-if)
# ------------------------------------------------------------
def main():
    # Base data (edit these numbers freely)
    base = {
        "T": [1, 2, 3, 4],
        "demand": {1: 80, 2: 120, 3: 60, 4: 140},
        "cap":    {1: 100, 2: 100, 3: 100, 4: 100},
        "c_prod": 5,
        "c_hold": 1,
        "c_unmet": 20,
        "c_setup": 200,
        "min_run": 30,
        "init_inv": 0
    }

    scenarios = [
        ("BASE", base),

        # Demand surge in period 2
        ("HIGH_DEMAND_T2", {**base, "demand": {1: 80, 2: 160, 3: 60, 4: 140}}),

        # Capacity drop (like “truck broken” but for factory)
        ("CAP_DROP_T4", {**base, "cap": {1: 100, 2: 100, 3: 100, 4: 70}}),

        # Bigger setup cost -> pushes fewer startups, more batching
        ("SETUP_MORE_EXPENSIVE", {**base, "c_setup": 400}),
    ]

    print("\n=== MILP RESULTS (real setup binaries) ===")
    for name, data in scenarios:
        res = solve_scenario(data, relax_for_duals=False, solver_name="glpk")
        print(f"\n--- {name} ---")
        print("Total cost:", round(res["total_cost"], 2))
        print("prod :", res["prod"])
        print("inv  :", res["inv"])
        print("unmet:", res["unmet"])
        print("setup:", {t: round(res['setup'][t], 3) for t in res["setup"]})

    print("\n=== RELAXED RESULTS (for shadow price intuition) ===")
    # One run to see "value of +1 capacity" via duals
    relaxed = solve_scenario(base, relax_for_duals=True, solver_name="glpk")
    print("\n--- BASE (RELAXED) ---")
    print("Total cost:", round(relaxed["total_cost"], 2))
    print("Capacity shadow prices (duals):", relaxed["duals_capacity"])
    print("Interpretation: if dual at t=2 is -3, then +1 capacity at t=2 reduces cost by ~3.")

if __name__ == "__main__":
    main()


=== MILP RESULTS (real setup binaries) ===

--- BASE ---
Total cost: 2860.0
prod : {1: 100.0, 2: 100.0, 3: 100.0, 4: 100.0}
inv  : {1: 20.0, 2: 0.0, 3: 40.0, 4: 0.0}
unmet: {1: -0.0, 2: -0.0, 3: -0.0, 4: 0.0}
setup: {1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0}

--- HIGH_DEMAND_T2 ---
Total cost: 3660.0
prod : {1: 100.0, 2: 100.0, 3: 100.0, 4: 100.0}
inv  : {1: 20.0, 2: 0.0, 3: 40.0, 4: 0.0}
unmet: {1: -0.0, 2: 40.0, 3: -0.0, 4: 0.0}
setup: {1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0}

--- CAP_DROP_T4 ---
Total cost: 3310.0
prod : {1: 100.0, 2: 100.0, 3: 100.0, 4: 70.0}
inv  : {1: 20.0, 2: 0.0, 3: 40.0, 4: 0.0}
unmet: {1: -0.0, 2: -0.0, 3: -0.0, 4: 30.0}
setup: {1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0}

--- SETUP_MORE_EXPENSIVE ---
Total cost: 3660.0
prod : {1: 100.0, 2: 100.0, 3: 100.0, 4: 100.0}
inv  : {1: 20.0, 2: 0.0, 3: 40.0, 4: 0.0}
unmet: {1: -0.0, 2: -0.0, 3: -0.0, 4: 0.0}
setup: {1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0}

=== RELAXED RESULTS (for shadow price intuition) ===

--- BASE (RELAXED) ---
Total cost: 2860.0
Ca